In [1]:
import numpy as np

nav_data = np.load(f'processed_nav_data.npy')
nav_labels = np.load(f'nav_data_labels.npy')
navs_columns = np.load('navs_columns_metadata.npy')

In [2]:
import pickle
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix

foldsNum = 4
kf = StratifiedKFold(n_splits=foldsNum, shuffle = True)
clf = svm.SVC(kernel = 'linear', class_weight='balanced')
acc = 0
n_classes = len(np.unique(nav_labels))
recall = np.zeros(n_classes)
precision = np.zeros(n_classes)
for train_index, test_index in kf.split(nav_data, nav_labels):
    
    # Training phase
    x_train = nav_data[train_index, :]
    y_train = nav_labels[train_index]
    clf.fit(x_train, y_train)

    # Test phase
    x_test = nav_data[test_index, :]
    y_test = nav_labels[test_index]    
    y_pred = clf.predict(x_test)
    # Calculate confusion matrix and model performance
    cm = confusion_matrix(y_test, y_pred)
    print('Confusion matrix\n', cm)

    accSum = 0
    for i in range(n_classes):
        accSum += cm[i,i]
    acc += accSum / len(y_test)

    for i in range(n_classes):
        innerSum = 0
        for y in range(n_classes):
            innerSum += cm[i,y]
        recall[i] += cm[i,i]/innerSum

    for i in range(n_classes):
        innerSum = 0
        for y in range(n_classes):
            innerSum += cm[y,i]
        precision[i] += cm[i,i]/innerSum

# Print results
acc = acc/foldsNum
print('Acc: ', acc)
precision = precision/foldsNum
print('Precision: ', precision)
recall = recall/foldsNum
print('Recall: ', recall)


with open(f'nav_svm_rbf.pkl', 'wb') as f:
    pickle.dump(clf, f)

Confusion matrix
 [[ 18  64]
 [  0 131]]
Confusion matrix
 [[ 24  57]
 [  0 131]]
Confusion matrix
 [[ 20  61]
 [  0 131]]
Confusion matrix
 [[ 31  50]
 [  1 130]]
Acc:  0.7255901762777925
Precision:  [0.9921875  0.69327932]
Recall:  [0.28635953 0.9980916 ]
